model 





In [1]:
import keras
#from cnnmodel import model
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import os, random, cv2
from keras.utils import to_categorical

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def createFileList(myDir):
    format = ('.tiff','.jpg','.png','.PNG')
    fileList = []
    print(os.walk(myDir, topdown=False))
    for root, dirs, files in os.walk(myDir, topdown=False):
        
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList 

In [3]:
classes={0:'angry', 1:'fear', 2:'happy', 3:'neutral', 4:'sad', 5:'surprise'}




batch_size = 32
epochs = 60

train_x = [] #(10000,1,227,227) (1,227,227)에 대해 append
train_y = [] #(10000, 6) 


#train_dir에 적용
#e.g. case sad
#path 읽어 = [(1,227,227)만들기 |  파싱하여 label 얻기] 

#train_x.append(q)
'''
z = load_img(path = './data/train/sad/KA.SA2.34.tiff', grayscale=True, target_size=(227,227),
    interpolation='nearest')
q=np.asarray(z).astype('float32')
print(np.asarray([q]).shape)
q = np.asarray([q])
'''

fileList = []
fileList = createFileList('./data/train')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    train_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    train_y.append(to_categorical(t, 6))

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
print(train_x.shape)   
print(train_y.shape)   

train_len = train_x.shape[0]
print(train_len)

<generator object walk at 0x7f7b590c6e08>
(1992, 1, 227, 227)
(1992, 6)
1992


In [4]:
#이상  train_x/ train_y 구성 끝

#val_x/ val_y 도 일단 똑같이 구성

val_x = []
val_y = []


fileList = createFileList('./data/valid')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    val_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    val_y.append(to_categorical(t, 6))

val_x = np.asarray(val_x)
val_y = np.asarray(val_y)
print(val_x.shape)   
print(val_y.shape)   

val_len = train_x.shape[0]


<generator object walk at 0x7f7b590c6f68>
(199, 1, 227, 227)
(199, 6)


In [5]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=10,
    shear_range=0.1,
    fill_mode = 'constant',
    cval = 0,
    horizontal_flip=True)

print(train_x.shape)
train_x_moveaxis = np.moveaxis(train_x, 1, 3)  
print(train_x_moveaxis.shape)
train_datagen.fit(train_x_moveaxis)

train_x_mean = train_datagen.mean
print(train_x_mean)

(1992, 1, 227, 227)
(1992, 227, 227, 1)
[[[118.80855]]]


In [6]:
print(val_x.shape)

(199, 1, 227, 227)


In [7]:
val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
val_x_moveaxis = np.moveaxis(val_x, 1, 3)  
val_datagen.fit(val_x_moveaxis)
#val_datagen.mean = x꺼 배끼끼
print(val_datagen.mean)
val_datagen.mean = train_x_mean
print(val_datagen.mean)

[[[118.370995]]]
[[[118.80855]]]


In [ ]:
test_x = []
test_y = []


fileList = createFileList('./data/test')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    test_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    test_y.append(to_categorical(t, 6))

test_x = np.asarray(test_x)
test_y = np.asarray(test_y)
print(test_x.shape)   
print(test_y.shape)   

test_len = train_x.shape[0]


test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
test_x_moveaxis = np.moveaxis(test_x, 1, 3)  
test_datagen.fit(test_x_moveaxis)
#val_datagen.mean = x꺼 배끼끼
print(test_datagen.mean)
test_datagen.mean = train_x_mean
print(test_datagen.mean)

In [10]:
#sequential model definition
#model.compile
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.layers.normalization import BatchNormalization


def model_generate():
    img_rows, img_cols = 227, 227
    model = Sequential()
    model.add(Convolution2D(36, 11, 11, border_mode='valid',
                            input_shape=(img_rows, img_cols,1)))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), dim_ordering='tf'))
    model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf')) 
    
    model.add(Convolution2D(96, 5, 5))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf')) 
    
    model.add(Convolution2D(144, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
     
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(Convolution2D(96, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
     
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
     
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))
    
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))
     
      
    model.add(Dense(6))
    model.add(BatchNormalization())
      
    model.add(Activation('softmax'))

    ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-08)
    #model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy',
                  optimizer=ada,
                  metrics=['accuracy'])
    model.summary()
    return model

model = model_generate()


filepath='model_8.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (11, 11), input_shape=(227, 227,..., padding="valid")`
  
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
  app.launch_new_instance()
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(2, 2), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (5, 5))`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launc

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 217, 217, 36)      4392      
_________________________________________________________________
batch_normalization_9 (Batch (None, 217, 217, 36)      144       
_________________________________________________________________
p_re_lu_8 (PReLU)            (None, 217, 217, 36)      1695204   
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 221, 221, 36)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 109, 109, 36)      0         
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin (None, 111, 111, 36)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 107, 107, 96)      86496     
__________

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`


In [11]:
model.fit_generator(train_datagen.flow(train_x_moveaxis, train_y, batch_size =batch_size),
                    #steps_per_epoch = train_len/batch_size*12,
                    steps_per_epoch = train_len/batch_size*4,
                    epochs = epochs,
                    validation_data = val_datagen.flow(val_x_moveaxis,val_y,batch_size = batch_size),
                    callbacks = [checkpointer])

#l = train_datagen.flow(train_x_moveaxis, train_y, batch_size =batch_size)

#next(l)[0].shape

Epoch 1/60
249/249 [==============================] - 73s 294ms/step - loss: 0.9229 - acc: 0.7411 - val_loss: 0.5711 - val_acc: 0.9095

Epoch 00001: saving model to model_8.hdf5
Epoch 2/60
249/249 [==============================] - 72s 290ms/step - loss: 0.6753 - acc: 0.8753 - val_loss: 0.5461 - val_acc: 0.9397

Epoch 00002: saving model to model_8.hdf5
Epoch 3/60
249/249 [==============================] - 73s 291ms/step - loss: 0.5813 - acc: 0.9241 - val_loss: 0.4513 - val_acc: 0.9799

Epoch 00003: saving model to model_8.hdf5
Epoch 4/60
249/249 [==============================] - 72s 291ms/step - loss: 0.5200 - acc: 0.9544 - val_loss: 0.4815 - val_acc: 0.9950

Epoch 00004: saving model to model_8.hdf5
Epoch 5/60
249/249 [==============================] - 72s 290ms/step - loss: 0.4897 - acc: 0.9629 - val_loss: 0.4256 - val_acc: 0.9849

Epoch 00005: saving model to model_8.hdf5
Epoch 6/60
249/249 [==============================] - 72s 291ms/step - loss: 0.4509 - acc: 0.9747 - val_loss: 

Epoch 47/60
249/249 [==============================] - 73s 291ms/step - loss: 0.1441 - acc: 0.9969 - val_loss: 0.1079 - val_acc: 1.0000

Epoch 00047: saving model to model_8.hdf5
Epoch 48/60
249/249 [==============================] - 72s 290ms/step - loss: 0.1390 - acc: 0.9985 - val_loss: 0.1049 - val_acc: 1.0000

Epoch 00048: saving model to model_8.hdf5
Epoch 49/60
249/249 [==============================] - 72s 291ms/step - loss: 0.1316 - acc: 0.9995 - val_loss: 0.1161 - val_acc: 1.0000

Epoch 00049: saving model to model_8.hdf5
Epoch 50/60
249/249 [==============================] - 72s 291ms/step - loss: 0.1327 - acc: 0.9979 - val_loss: 0.1160 - val_acc: 1.0000

Epoch 00050: saving model to model_8.hdf5
Epoch 51/60
249/249 [==============================] - 72s 291ms/step - loss: 0.1306 - acc: 0.9989 - val_loss: 0.0947 - val_acc: 1.0000

Epoch 00051: saving model to model_8.hdf5
Epoch 52/60
249/249 [==============================] - 72s 290ms/step - loss: 0.1290 - acc: 0.9982 - val_

In [ ]:
#classes={0:'angry', 1:'fear', 2:'happy', 3:'neutral', 4:'sad', 5:'surprise'}
#vd = val_datagen.flow(val_x_moveaxis,val_y,batch_size = 1)
#sample = vd.next()
#sample_x,sample_y = sample
#print(sam)
from keras.models import load_model

model = load_model(filepath)
"""
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
#print("0hi")
#print(test_datagen.mean)
test_datagen.fit(val_x_moveaxis[:10])
#print(test_datagen.mean)
test_datagen.mean = val_datagen.mean
#print(test_datagen.mean)
"""


e = model.predict_generator(test_datagen.flow(test_x_moveaxis,test_y,batch_size = 1,shuffle= False))
print(val_datagen.mean)
print([np.argmax(pre) for pre in e])
#print(val_x_moveaxis[0])
#print(val_y)
#print(e)
print([np.argmax(x) for x in test_y])

In [ ]:
correct = 0
for x,y in zip(e,test_y):
    if np.argmax(x) == np.argmax(y):
        correct+=1
    else: continue
        
print(float(correct)/len(test_y)*100)

A total data # * steps for epochs / batchsize  ==  172 
Therefore, 172 * 32(batchsize) == A total data #

In [ ]:

valid_x = [] #(10000,1,227,227) (1,227,227)에 대해 append
valid_y = [] #(10000, 6) 


fileList = []
fileList = createFileList('./data/valid')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    valid_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    valid_y.append(to_categorical(t, 6))

    
valid_x = np.asarray(valid_x)
valid_y = np.asarray(valid_y)
print(valid_x.shape)   
print(valid_y.shape)   
valid_x_moveaxis = np.moveaxis(valid_x, 1, 3)  
valid_x_moveaxis.shape


In [ ]:


valid_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
#print("0hi")
#print(test_datagen.mean)
valid_datagen.fit(valid_x_moveaxis[:])
print(valid_datagen.mean)
valid_datagen.mean = [[[0]]]
print(valid_datagen.mean)



e1 = model.predict_generator(valid_datagen.flow(valid_x_moveaxis,valid_y,batch_size = 1,shuffle= False))
print(valid_datagen.mean)
print([np.argmax(pre) for pre in e1])
#print(val_x_moveaxis[0])
#print(val_y)
print(e1)
print([np.argmax(x) for x in valid_y])



correct = 0
for x,y in zip(e1,valid_y):
    if np.argmax(x) == np.argmax(y):
        correct+=1
    else: continue
        
print(float(correct)/len(valid_y)*100)
